In [ ]:
import os
CWD = os.getcwd()

import sys
sys.path = [CWD+"/../.."]+sys.path

import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.metricBundles as metricBundles
from rubin_sim.maf.metrics import QSONumberCountsMetric

In [ ]:
from opsimUtils import *
from ExgalM5_with_cuts_AGN import ExgalM5_with_cuts_AGN
from script_utils import find_completed_runs

In [ ]:
#We will use the same slicer and constraint for each metric. 
NSIDE=64
slicer = slicers.HealpixSlicer(nside=NSIDE)
constraint = 'note not like "DD%"' #remove DDFs

In [ ]:
#Set the metrics. Use the i-band for both. 
filter = 'i'
metric_em5  = ExgalM5_with_cuts_AGN(lsstFilter=filter)
metric_nqso = QSONumberCountsMetric(lsstFilter=filter)

In [ ]:
#Setup the metric bundles.
constraint_use = 'filter = "{}" and '.format(filter) + constraint
EM5 = metricBundles.MetricBundle(metric_em5 , slicer, constraint_use)
NQSO = metricBundles.MetricBundle(metric_nqso, slicer, constraint_use)

In [ ]:
#Setup the bundle dictionary
bundleDict = dict()
bundleDict['EM5'] = EM5
bundleDict['NQSO'] = NQSO

In [ ]:
#Setup the output folders.
folder_mafoutput = "NQSO_test_{0:d}".format(NSIDE)
cwd = os.getcwd()
outDir = cwd+'/../../scratch/MAFOutput/{}'.format(folder_mafoutput)
if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

resultDbPath  = cwd+'/../../scratch/MAFOutput/{}'.format(folder_mafoutput)
metricDataPath = cwd+'/../../scratch/MAFOutput/{}/MetricData/'.format(folder_mafoutput)

In [ ]:
dbDir = '/sims_maf/fbs_1.5/baseline/'
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)
dbRuns = show_opsims(dbDir)
for run in dbRuns:
    EM5.setRunName(run)
    NQSO.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                        opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()